In [1]:
#re.findall('\(.*?\)',b1)
#re.findall('\(.*?\)',f1)

In [199]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import copy

from goodreads import client
gc = client.GoodreadsClient('9addHIFaPJmit7dzC5ZA', 'iDXvx13Rnt1iJkWOxRkSh9wLnbFWmMNgCUMFLJmLPo')

from imdb import IMDb

In [3]:
wikipages = ['https://en.wikipedia.org/wiki/List_of_children%27s_books_made_into_feature_films?oldformat=true',
             'https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(0%E2%80%939,_A%E2%80%93C)?oldformat=true',
             'https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(D%E2%80%93J)?oldformat=true',
             'https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(K%E2%80%93R)?oldformat=true',
             'https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(S%E2%80%93Z)?oldformat=true'
            ]

In [4]:
def add_to_rows(page_soup):
    """
    Method extracts row elements from the soup tags of individual 
    list pages (A - F, H - J, etc.) and appends them to the `rows` 
    list.
    
    Args:
        page_soup: (BeautifulSoup tag) soup for list page.
    """
    
    global rows
    
    tables = page_soup.find_all('table', {'class': 'wikitable'})

    for alphabet in tables:
        rows += alphabet.findAll('tr')
    
    print("Tables added: {}, Rows added: {}".format(len(tables), len(rows)))

In [166]:
def working_wiki_link(wiki_link):
    """
    Converts relative URLs to global URLs.
    """
    if '/wiki/' in wiki_link:
        return 'https://en.wikipedia.org' + wiki_link
    return wiki_link

In [152]:
def get_adaptations(book_title, href_list):
    """
    Parses through list of movies associated with a specific `book_title`,
    adds movies to `movie_book_dict`, and returns list of movies.
    Adds year to the movie title if information available. 
    
    Args:
        book_title: (str) name of original book.
        href_list: (list) Result of findAll('a') on the movies column.
        
    Returns:
        adaptations: List of movie title names.
    """
    adaptations = []
    
    if len(href_list) == 1:
        adaptations.append(href_list[0]['title'])
        return adaptations
        
    for index, tag in enumerate(href_list):
        year_search = re.search(r'\d\d\d\d', tag['href'])
        
        if year_search is not None:  #There is a 4 digit number in the title
            year_search = year_search.group()
        
            if year_search not in tag['title']:
                version = tag['title'] + ' (' + year_search + ')'
            else:
                version = tag['title']
            
            adaptations.append(version)
            movie_book_dict[working_wiki_link(tag['href'])] = {'movie_title': version, 
                                                              'book': book_title,
                                                              'imdb_no': '', 'usable': ''}
        
        else:
            version = tag['title']
            adaptations.append(version)
            movie_book_dict[working_wiki_link(tag['href'])] = {'movie_title': version, 
                                                              'book': book_title,
                                                              'imdb_no': '', 'usable': ''}

    
    return adaptations

def clean_adaptations(adaptations):
    """
    Makes sure the list of adaptations doesn't contain TV series or miniseries.
    """
    
    cleaned = []
    
    for title in adaptations:
        if 'TV' in title or 'miniseries' in title:
            continue
        if 'page does not exist' in title:
            continue
        else:
            cleaned.append(title)
            
    return cleaned

In [26]:
def add_to_book_film_dict():
    """
    Method parses newly updated rows for book/film title, book author,
    number of adaptions, and list of adaptations, and adds them to the
    `book_film_dict` as a nested dictionary. 
    global sum_adaptations
    """
    global sum_adaptations
    
    for row in rows:
        cols = row.findAll('td')

        try:
            if (len(cols[0].findAll('a')) == 2) and (cols[1].find('a') is not None):
                series = cols[0].text.replace('\n', '').split(',')[0]

                book_title = cols[0].findAll('a')[0]['title']
                author = cols[0].findAll('a')[-1]['title']

                book_wiki_url = working_wiki_link(cols[0].findAll('a')[0]['href'])

                try:
                    adaptations = clean_adaptations(get_adaptations(book_title, cols[1].findAll('a')))
                except KeyError:
                    adaptations = []
                    pass

                sum_adaptations += len(adaptations)

                book_film_dict_all[series] = {'author': author,
                                            'book_title': book_title,
                                            'book_wiki_url': book_wiki_url,
                                            'count': len(adaptations),
                                            'adaptations': adaptations,
                                            'isbn': '', 'oclc':'', 'valid_identifier': ''}

        except IndexError or KeyError:
            pass
        
    print("No. of rows/original books in dict: {}, No. of adaptations: {} \n"
          .format(len(book_film_dict_all), sum_adaptations))

In [153]:
rows = []
book_film_dict_all = {}
movie_book_dict = {}
sum_adaptations = 0

for url in wikipages:
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'lxml')
    
    add_to_rows(soup)
    add_to_book_film_dict()

Tables added: 25, Rows added: 376
No. of rows/original books in dict: 259, No. of adaptations: 754 

Tables added: 3, Rows added: 798
No. of rows/original books in dict: 553, No. of adaptations: 2046 

Tables added: 7, Rows added: 1245
No. of rows/original books in dict: 813, No. of adaptations: 3917 

Tables added: 8, Rows added: 1662
No. of rows/original books in dict: 1056, No. of adaptations: 6426 

Tables added: 7, Rows added: 2034
No. of rows/original books in dict: 1237, No. of adaptations: 9466 



In [28]:
len_initial = len(book_film_dict_all)
len_initial

1237

In [30]:
movie_book_dict['https://en.wikipedia.org/wiki/102_Dalmatians'], movie_book_dict['https://en.wikipedia.org/wiki/3_Godfathers']

({'movie_title': '102 Dalmatians', 'book': 'The Hundred and One Dalmatians'},
 {'movie_title': '3 Godfathers', 'book': 'Three Godfathers'})

## Don't touch above

In [20]:
def get_isbn(wiki_url):
    """
    Parses the `infobox vcard` element of the wikipedia page of a book, 
    and returns the 10-digit or 13-digit ISBN number (if found). 
    
    Args:
        wiki_url (str): Valid (not local) wikipedia link.

    Returns:
        isbn: Returns ISBN code is parsed correctly, 'broken' if formmated 
        incorrectly or otherwise. The code is de-hyphentated for later pipeline
        efficiency. 
 
    """
    try:
        page = requests.get(wiki_url).text
        soup = BeautifulSoup(page, 'lxml')
    except MissingSchema:
        return 'broken'  
    
    infobox = soup.find('table', {'class': 'infobox vcard'})
    
    if infobox is None:
        return 'broken'
    
    for row in infobox.findAll('tr'):
        #ISBN directly available
        if ('ISBN' in row.text):
            isbn = re.sub('[^0-9]','', row.findAll('a')[-1].text)
            return isbn
        
    #Nothing found directly on Infobox
    return 'broken' 

In [21]:
# Only to be used if get_isbn() returns 'not formatted correctly'
def get_oclc(wiki_url):
    """
    Parses the `infobox vcard` element of the wikipedia page of a book, 
    and returns the 10-digit or 13-digit OCLC number (if found). This function is a backup
    if get_isbn() returns 'broken', and must not be relied on. 
    
    Args:
        wiki_url (str): Valid (not local) wikipedia link.

    Returns:
        oclc: Returns OCLC code is parsed correctly, 'broken' if formmated 
        incorrectly or otherwise. 
 
    """
    
    try:
        page = requests.get(wiki_url).text
        soup = BeautifulSoup(page, 'lxml')
    except MissingSchema:
        return 'broken'

    infobox = soup.find('table', {'class': 'infobox vcard'})
    
    if infobox is None:
        return 'broken'
    
    for row in infobox.findAll('tr'):
        #only OCLC available
        if row.find('a', {'title': 'OCLC'}) is not None:
            return (row.find('td').text)
        
    #Nothing found directly on Infobox
    return 'broken'     

In [154]:
broken_count = 0
usable_count = 0
index = 0
book_film_dict = {}

for key in book_film_dict_all.keys():
    
    link = book_film_dict_all[key]['book_wiki_url']
    
    try:
        book_film_dict_all[key]['isbn'] = get_isbn(link)
    
        if book_film_dict_all[key]['isbn'] == 'broken':
            book_film_dict_all[key]['oclc'] = get_oclc(link)
            if book_film_dict_all[key]['oclc'] == 'broken':
                book_film_dict_all[key]['valid_identifer'] = False
                #Nothing changes, the book is unusable, and the 'usable' flag remains False
        else:
            book_film_dict_all[key]['valid_identifer'] = True
            book_film_dict[key] = copy.deepcopy(book_film_dict_all[key]) #Deepcopy 'true' values to another dictionary
            usable_count += 1
    except NameError:
        pass
        
    if (index % 100 == 0):
        broken_count = index - usable_count
        print("Usable books: {}, Broken books: {}, Total books: {}".format(usable_count, broken_count, index))
    
    index += 1
    
print("Usable books: {}, Broken books: {}, Total books: {}".format(usable_count, broken_count, len_initial))

Usable books: 0, Broken books: 0, Total books: 0
Usable books: 51, Broken books: 49, Total books: 100
Usable books: 99, Broken books: 101, Total books: 200
Usable books: 145, Broken books: 155, Total books: 300
Usable books: 175, Broken books: 225, Total books: 400
Usable books: 203, Broken books: 297, Total books: 500
Usable books: 245, Broken books: 355, Total books: 600
Usable books: 293, Broken books: 407, Total books: 700
Usable books: 334, Broken books: 466, Total books: 800
Usable books: 371, Broken books: 529, Total books: 900
Usable books: 406, Broken books: 594, Total books: 1000
Usable books: 444, Broken books: 656, Total books: 1100
Usable books: 486, Broken books: 714, Total books: 1200
Usable books: 501, Broken books: 714, Total books: 1237


In [155]:
len(book_film_dict), len(movie_book_dict)

(501, 1435)

In [157]:
safe_movie_book_dict = copy.deepcopy(movie_book_dict)
id(safe_movie_book_dict), id(movie_book_dict)

(4900115728, 4847924352)

we lose a bunch of old books, but it's okay since inflation would have destroyed old movie sales as well. i guess it wokrs out

## Don't touch above

In [62]:
book_film_dict['Dreamcatcher (2001)']

{'author': 'Stephen King',
 'book_title': 'Dreamcatcher (novel)',
 'book_wiki_url': 'https://en.wikipedia.org/wiki/Dreamcatcher_(novel)',
 'count': 1,
 'adaptations': ['Dreamcatcher (2003 film)'],
 'isbn': '9780743211383',
 'oclc': '',
 'valid_identifier': '',
 'valid_identifer': True}

In [63]:
book_film_dict["Gulliver's Travels (1726)"]

KeyError: "Gulliver's Travels (1726)"

In [64]:
book_film_dict_all["Gulliver\'s Travels (1726)"]

{'author': 'Jonathan Swift',
 'book_title': "Gulliver's Travels",
 'book_wiki_url': 'https://en.wikipedia.org/wiki/Gulliver%27s_Travels',
 'count': 7,
 'adaptations': ['Le Voyage de Gulliver à Lilliput et chez les géants (1902 film)',
  'The New Gulliver',
  "Gulliver's Travels (1939 film)",
  'The Three Worlds of Gulliver',
  "Gulliver's Travels Beyond the Moon",
  "Gulliver's Travels (1977 film)",
  "Gulliver's Travels (2010 film)"],
 'isbn': 'broken',
 'oclc': 'broken',
 'valid_identifier': '',
 'valid_identifer': False}

In [65]:
desired_metadata = ['average_rating', 
                    'publication_date',
                    'rating_dist',
                    'ratings_count', 'format',
                    'text_reviews_count',
                    'title', 'language_code']

def get_goodreads_data(gc_code):
    """
    Add additional review and ratings related meta_data from the GoodReads API.
    
    Args:
        gc_code: Strictly Goodreads code. Can be 10-digit or 13-digit.
        
    Returns:
        gr_metadata: Dict with additional metadata.
    """
    
    gr_metadata = {}
    try:
        book = gc.book(gc_code)
    except NameError:
        return gr_metadata
    except ExpatError:
        return gr_metadata
    
    for attrs in book.__dict__['_book_dict']:
        if attrs in desired_metadata:
            gr_metadata['gc_' + attrs] = book.__dict__['_book_dict'][attrs]
            
    return gr_metadata

In [207]:
book_metadata_wanted = ['identifiers', 'number_of_pages', 'subject_places', 'subjects', 'publish_date', 'publish_places']

def update_book_metadata(book_title):
    """
    Uses the OpenLibrary and Goodreads APIs (when possible) to 
    add metadata about the books to the `book_film_dict`.
    
    Args: 
        book_title: (str) used when merging back into `book_film_dict`.
    
    Returns:
        updated: (bool) True if metadata updated, False otherwise.
    
    
    """
    metadata = {}
    all_metadata = {}
    
    
    if book_film_dict[book_title]['valid_identifer'] is False:
        #API can't be accessed
        metadata['metadata_updated'] = False
        book_film_dict[book_title].update(metadata)
        return
    
    elif book_film_dict[book_title]['isbn'] != '':
        code = book_film_dict[book_title]['isbn']
        curl = 'https://openlibrary.org/api/books?bibkeys=ISBN:{}&jscmd=data&format=json'.format(code)
        all_metadata = requests.get(curl).json()
    else:
        code = book_film_dict[book_title]['oclc']
        curl = 'https://openlibrary.org/api/books?bibkeys=OCLC:{}&jscmd=data&format=json'.format(code)
        all_metadata = requests.get(curl).json()
        
    if len(all_metadata) > 0:
        key = list(all_metadata.keys())[0]
        nested_keys = list(all_metadata[key].keys())
        
        for data_point in book_metadata_wanted:
            if data_point in nested_keys:
                metadata[data_point] = all_metadata[key][data_point]
    
        if 'subject_places' in nested_keys:
            metadata['subject_places'] = [place['name'][:30] for place in metadata['subject_places']]
        if 'subjects' in nested_keys:
            metadata['subjects'] = [subject['name'][:30] for subject in metadata['subjects']]
        if 'publish_places' in nested_keys:
            metadata['publish_places'] = [place['name'][:30] for place in metadata['publish_places']]            
        
        if 'identifiers' in nested_keys:
            if 'goodreads' in list(metadata['identifiers'].keys()):
                metadata['goodreads'] = metadata['identifiers']['goodreads'][0]
            metadata.pop('identifiers')
            
        if 'goodreads' in metadata.keys():
            metadata.update(get_goodreads_data(metadata['goodreads']))
            
        metadata['metadata_updated'] = True
        book_film_dict[book_title].update(metadata)
        return
    
    else:
        metadata['metadata_updated'] = False
        book_film_dict[book_title].update(metadata)
        return


In [67]:
update_book_metadata('Dreamcatcher (2001)')
book_film_dict['Dreamcatcher (2001)']

{'author': 'Stephen King',
 'book_title': 'Dreamcatcher (novel)',
 'book_wiki_url': 'https://en.wikipedia.org/wiki/Dreamcatcher_(novel)',
 'count': 1,
 'adaptations': ['Dreamcatcher (2003 film)'],
 'isbn': '9780743211383',
 'oclc': '',
 'valid_identifier': '',
 'valid_identifer': True,
 'number_of_pages': 620,
 'subject_places': ['Maine', 'Derry', 'Hole-in-the-Wall', 'Jefferson Tract'],
 'subjects': ['Internet Archive Wishlist',
  'male friendship',
  'life on other planets',
  'telepathy',
  'Down Syndrome',
  'psychological fiction',
  'extraterrestrial beings',
  'hunting',
  'friendship',
  'In library',
  'Accessible book',
  'Fiction',
  'Human-alien encounters',
  'Protected DAISY',
  'Fear',
  'Fantasy Fiction',
  'Roman fantastique',
  'Audiobooks',
  'hunting stories',
  'camping',
  'Male friendship'],
 'publish_date': '2001',
 'publish_places': ['New York, USA'],
 'goodreads': '441240',
 'gc_title': 'Dreamcatcher',
 'gc_language_code': 'eng',
 'gc_average_rating': '3.62',
 

don't run below till the end

In [68]:
from xml.parsers.expat import ExpatError

metadata_update_count = 0

for index, key in enumerate(book_film_dict.keys()):
    
    if book_film_dict[key]['metadata_updated'] is True:
        metadata_update_count += 1
    
    if index % 75 == 0 and index != 0:
        print("Indexed: {}, Updated: {}, Usable: {}%"
              .format(index, metadata_update_count, round(100*metadata_update_count/index, 3)))

Indexed: 75, Updated: 59, Usable: 78.667%
Indexed: 150, Updated: 125, Usable: 83.333%
Indexed: 225, Updated: 186, Usable: 82.667%
Indexed: 300, Updated: 250, Usable: 83.333%
Indexed: 375, Updated: 319, Usable: 85.067%
Indexed: 450, Updated: 383, Usable: 85.111%


In [69]:
print("Done. Started with {} rows; now {} are usable: ~{}%."
    .format(len_initial, metadata_update_count, round(100*metadata_update_count/len_initial, 3)))

Done. Started with 1237 rows; now 429 are usable: ~34.681%.


In [72]:
book_film_dict['Grand Canary (1933)']

{'author': 'A. J. Cronin',
 'book_title': 'Grand Canary (novel)',
 'book_wiki_url': 'https://en.wikipedia.org/wiki/Grand_Canary_(novel)',
 'count': 1,
 'adaptations': ['Grand Canary (film)'],
 'isbn': '0450020479',
 'oclc': '',
 'valid_identifier': '',
 'valid_identifer': True,
 'number_of_pages': 223,
 'subject_places': ['Canary Islands',
  'Gran Canaria',
  'Gran Canaria (Canary Islands)'],
 'subjects': ['Fiction', 'Physicians', 'Fiction in English'],
 'publish_date': '1975',
 'publish_places': ['London'],
 'goodreads': '184807',
 'metadata_updated': True}

## Moving on to movies

In [76]:
movie_book_dict['https://en.wikipedia.org/wiki/1984_(1956_film)']

{'movie_title': '1984 (1956 film)', 'book': 'Nineteen Eighty-Four'}

In [134]:
def get_imdb_no(wiki_movie):
    """
    Returns movie's respective IMDB no (if found). 
    
    Args: 
        wiki_movie: (str) Wikipedia link of movie's article.
        
    Returns:
        imdb_no: (str) IMDB no. if found, 'broken' otherwise.
    """
    imdb_no = ''   
   
    try:
        page = requests.get(wiki_movie).text
        soup = BeautifulSoup(page, 'lxml')
    except errors:
        return 'broken'
    
    external_links = soup.find('a', href=re.compile("imdb.com/title"))
    
    try:
        imdb_no = external_links['href'].split('/')[4]
    except TypeError:
        return 'broken'
    
    return imdb_no
    
get_imdb_no('https://en.wikipedia.org/wiki/Charlotte%27s_Web_(2006_film)')
    

'tt0413895'

In [182]:
movie_book_dict = copy.deepcopy(safe_movie_book_dict)
len(movie_book_dict)

1435

In [195]:
from socket import gaierror

usable_count = 0
errors = (requests.exceptions.MissingSchema or 
          socket.gaierror or 
          KeyError or 
          requests.exceptions.ConnectionError or 
          requests.exceptions.NewConnectionError or 
          requests.exceptions.MaxRetryError)

for index, key in enumerate(movie_book_dict.keys()):
    movie_book_dict[key]['usable'] = False #default is false
    
    try:
        movie_book_dict[key]['imdb_no'] = get_imdb_no(key)
    except errors: 
        movie_book_dict[key]['imdb_no'] = 'broken'
    
    if movie_book_dict[key]['imdb_no'] != 'broken':
        movie_book_dict[key]['usable'] = True
        usable_count += 1
           
    if (index + 1) % 150 == 0:
        print('Index: {}, Usable: {} --> {}%'
              .format(index + 1, usable_count, round(usable_count*100/(index+1), 3)))    

Index: 150, Usable: 142 --> 94.667%
Index: 300, Usable: 283 --> 94.333%
Index: 450, Usable: 427 --> 94.889%
Index: 600, Usable: 567 --> 94.5%
Index: 750, Usable: 712 --> 94.933%
Index: 900, Usable: 855 --> 95.0%
Index: 1050, Usable: 995 --> 94.762%
Index: 1200, Usable: 1139 --> 94.917%
Index: 1350, Usable: 1269 --> 94.0%


In [198]:
movie_book_dict['https://en.wikipedia.org/wiki/1984_(1956_film)']

{'movie_title': '1984 (1956 film)',
 'book': 'Nineteen Eighty-Four',
 'imdb_no': 'tt0048918',
 'usable': True}

In [235]:
movie_metadata_wanted = ['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 
                         'English', 'imdbRating', 'imdbVotes']

def add_imdb_metadata(movie_wiki_url):
    """
    
    """
    metadata = {}
    all_metadata = {}
    imdb_no = movie_book_dict[movie_wiki_url]['imdb_no']
    
    all_metadata = requests.get('http://www.omdbapi.com/?i={}&apikey=2cb213a7'.format(imdb_no)).json()
    
    for key in all_metadata.keys():
        if key in movie_metadata_wanted:
            metadata['imdb_' + key.lower()] = all_metadata[key]
            
        if 'Actors' in all_metadata.keys():
            metadata['imdb_actors'] = get_imdb_actors(all_metadata['Actors'])
        if 'Writer' in all_metadata.keys():
            metadata['imdb_writers'] = get_imdb_writers(all_metadata['Writer'])
            
    metadata['metadata_updated'] = True
    movie_book_dict[movie_wiki_url].update(metadata)

In [226]:
def get_imdb_actors(actor_string):
    actors = actor_string.rstrip('.').split(',')
    actors = [a.lower().strip() for a in actors]    
    return actors

def get_imdb_writers(writer_string):
    writers = writer_string.rstrip('.').split(',')
    writers = [re.sub(r'\([^()]*\)', '', w) for w in writers]
    writers = [w.lower().strip() for w in writers] 
    return writers

In [228]:
movie_book_dict['https://en.wikipedia.org/wiki/Jaws_(film)']

{'movie_title': 'Jaws (film)',
 'book': 'Jaws (novel)',
 'imdb_no': 'tt0073195',
 'usable': True}

In [229]:
add_imdb_metadata('https://en.wikipedia.org/wiki/Jaws_(film)')
movie_book_dict['https://en.wikipedia.org/wiki/Jaws_(film)']

{'movie_title': 'Jaws (film)',
 'book': 'Jaws (novel)',
 'imdb_no': 'tt0073195',
 'usable': True,
 'imdb_title': 'Jaws',
 'imdb_actors': ['roy scheider',
  'robert shaw',
  'richard dreyfuss',
  'lorraine gary'],
 'imdb_writers': ['peter benchley', 'carl gottlieb', 'peter benchley'],
 'imdb_year': '1975',
 'imdb_rated': 'PG',
 'imdb_released': '20 Jun 1975',
 'imdb_runtime': '124 min',
 'imdb_genre': 'Adventure, Drama, Thriller',
 'imdb_imdbrating': '8.0',
 'imdb_imdbvotes': '511,159',
 'metadata_updated': True}

In [239]:
updated_count = 0
for index, key in enumerate(movie_book_dict.keys()):
    
    if movie_book_dict[key]['usable'] is True:
        try:
            add_imdb_metadata(key)
        except json.JSONDecodeError:
            movie_book_dict[key]['usable'] = False
            movie_book_dict[key]['imdb_no'] = 'broken'
            updated_count -= 1
            pass
        
        updated_count += 1
        
    if index % 150 == 0 and index != 0:
        print('Index: {}, Updated: {} --> {}%'.format(index, updated_count, updated_count * 100/index))
        
print('Index: {}, Updated: {} --> {}%'.format(index, updated_count, updated_count * 100/index))

Index: 150, Updated: 143 --> 95.33333333333333%
Index: 300, Updated: 284 --> 94.66666666666667%
Index: 450, Updated: 428 --> 95.11111111111111%
Index: 600, Updated: 568 --> 94.66666666666667%
Index: 750, Updated: 713 --> 95.06666666666666%
Index: 900, Updated: 856 --> 95.11111111111111%
Index: 1050, Updated: 996 --> 94.85714285714286%
Index: 1200, Updated: 1139 --> 94.91666666666667%
Index: 1350, Updated: 1269 --> 94.0%
Index: 1434, Updated: 1351 --> 94.21199442119944%


In [240]:
movie_book_dict['https://en.wikipedia.org/wiki/It_(2017_film)']

{'movie_title': 'It (2017 film)',
 'book': 'It (novel)',
 'imdb_no': 'tt1396484?d=inprod',
 'usable': True,
 'metadata_updated': True}

In [241]:
movie_book_dict['ht']

{'movie_title': 'For Whom the Bell Tolls (film)',
 'book': 'For Whom the Bell Tolls',
 'imdb_no': 'tt0035896',
 'usable': True,
 'imdb_title': 'For Whom the Bell Tolls',
 'imdb_actors': ['gary cooper',
  'ingrid bergman',
  'akim tamiroff',
  'arturo de córdova'],
 'imdb_writers': ['dudley nichols', 'ernest hemingway'],
 'imdb_year': '1943',
 'imdb_rated': 'Passed',
 'imdb_released': '28 Apr 1944',
 'imdb_runtime': '170 min',
 'imdb_genre': 'Adventure, Drama, History, Romance, War',
 'imdb_imdbrating': '6.9',
 'imdb_imdbvotes': '7,624',
 'metadata_updated': True}

## Bringing in to pandas

In [ ]:
book_film = pd.DataFrame()
rows = []

for index, key in enumerate(book_film_dict.keys()):
    get_book_metadata(key)
    
    for movie in book_film_dict[key]['adaptations']:
        row = [key, book_film_dict[key]['author'], 
               movie, 
               book_film_dict[key]['count'], 
               book_film_dict[key]['isbn'],
               book_film_dict[key]['oclc'],
               book_film_dict[key]['usable'],
        rows.append(row)
        
    if index > 20:
        break
    
book_film = pd.DataFrame(data=rows)
book_film.rename(columns={0: 'book_title', 
                          1: 'author', 
                          2: 'movie', 
                          3: 'total_ad_count', 
                          4: 'isbn', 5: 'oclc', 
                          6: 'usable'},
                inplace = True)

book_film.head()

In [244]:
boom = {'x': 2132, 'y': [1, 2, 4]}
cafe = {'x': 43243, 'y': 3}

hamos = {'f': boom, 's': cafe}
pd.Dataframe.from_dict(hamos)

AttributeError: module 'pandas' has no attribute 'Dataframe'

In [253]:
s = pd.DataFrame.from_dict(hamos)
type(s.iloc[1, 0])

list